In [1]:
import cv2
import numpy as np

from pathlib import Path


In [2]:
class ImageProcessor:
    def vertical(self, image):
        flipped_image = cv2.flip(image, 1)
        return flipped_image

    def horizontal(self, image):
        flipped_image = cv2.flip(image, 0)
        return flipped_image

    def gaussian_noise(self, image, mean=0, noise_percent=10):
        sigma = 255 * (noise_percent / 100)
        gaussian = np.random.normal(mean, sigma, image.shape).astype(np.float32)
        gaussian_image = cv2.add(image.astype(np.float32), gaussian)
        gaussian_image = np.clip(gaussian_image, 0, 255).astype(np.uint8)
        return gaussian_image

    def salt_pepper_noise(self, image, noise_percent=10, ratio=50):
        salt_pepper_image = np.copy(image)
        noise_percent, ratio = noise_percent / 100, ratio / 100
        salt_pepper_image = self._salt_pepper(image=image, noise_percent=noise_percent, ratio=ratio)
        return salt_pepper_image

    def adjust_contrast(self, image, adjust_percent=10):
        adjust_percent = adjust_percent/100
        adjusted_image = cv2.convertScaleAbs(image, alpha=adjust_percent)
        return adjusted_image

    def adjust_bright(self, image, adjust_percent):
        adjust_percent = int(255*(adjust_percent/100))
        adjusted_image = cv2.convertScaleAbs(image, beta=adjust_percent)
        return adjusted_image

    def _salt_pepper(self, image, noise_percent, ratio):
        num_pixels = image.shape[0] * image.shape[1]
        num_salt = np.ceil(noise_percent * num_pixels * ratio)
        num_pepper = np.ceil(noise_percent * num_pixels * (1.0 - ratio))
        salt = self._apply_noise(image, num_salt, 255)
        salt_pepper = self._apply_noise(salt, num_pepper, 0)
        return salt_pepper

    def _apply_noise(self, image, point, value):
        coords = [np.random.randint(0, i, int(point)) for i in image.shape[:2]]
        image[coords[0], coords[1], :] = value
        return image


In [3]:
class LabelProcessor:
    def vertical(self, labels):
        new_labels = self._apply_vertical_flip(labels=labels)
        return new_labels

    def horizontal(self, labels):
        new_labels = self._apply_horizontal_flip(labels=labels)
        return new_labels

    def _apply_vertical_flip(self, labels):
        new_labels = []
        for label in labels:
            cls, bbox, keypoints = self._parse_label(label=label)
            new_bbox = self._vertical_flip_bbox(bbox=bbox)
            new_keypoints = self._vertical_flip_keypoints(keypoints=keypoints)
            new_label = cls, new_bbox, new_keypoints
            new_labels.append(new_label)
        return new_labels

    def _apply_horizontal_flip(self, labels):
        new_labels = []
        for label in labels:
            cls, bbox, keypoints = self._parse_label(label=label)
            new_bbox = self._horizontal_flip_bbox(bbox=bbox)
            new_keypoints = self._horizontal_flip_keypoints(keypoints=keypoints)
            new_label = cls, new_bbox, new_keypoints
            new_labels.append(new_label)
        return new_labels

    def _parse_label(self, label):
        cls = int(label[0])
        bbox = list(label[1:5])
        keypoints = list(label[5:])
        return cls, bbox, keypoints

    def _vertical_flip_bbox(self, bbox):
        bbox[2] = 1 - bbox[2]
        return bbox

    def _vertical_flip_keypoints(self, keypoints):
        for i in range(0, len(keypoints), 3):
            keypoints[i] = 1 - keypoints[i]
        return keypoints

    def _horizontal_flip_bbox(self, bbox):
        bbox[1] = 1 - bbox[1]
        return bbox

    def _horizontal_flip_keypoints(self, keypoints):
        for i in range(1, len(keypoints), 3):
            keypoints[i] = 1 - keypoints[i]
        return keypoints


In [4]:
class DataAugment:
    def __init__(self, image_path, label_path, output_dir):
        self.image_path = Path(image_path)
        self.label_path = Path(label_path)
        self.image_suffix = self.image_path.suffix
        self.label_suffix = self.label_path.suffix

        self.output_dir = Path(output_dir)
        self.output_image_path = self.output_dir / 'images'
        self.output_label_path = self.output_dir / 'labels'
        self.output_image_path.mkdir(parents=True, exist_ok=True)
        self.output_label_path.mkdir(parents=True, exist_ok=True)

        self.image_processor = ImageProcessor()
        self.label_processor = LabelProcessor()

        self.current_image = cv2.imread(str(image_path))
        with open(label_path, 'r') as f:
            self.current_labels = [line.strip().split() for line in f.readlines()]

        self.filename_suffix = ""

    def vertical(self):
        self.current_image = self.image_processor.vertical(image=self.current_image)
        self.current_labels = self.label_processor.vertical(labels=self.current_labels)
        self.filename_suffix += f"_vert"
        return self

    def horizontal(self):
        self.current_image = self.image_processor.horizontal(image=self.current_image)
        self.current_labels = self.label_processor.horizontal(labels=self.current_labels)
        self.filename_suffix += f"_horiz"
        return self

    def gaussian_noise(self, noise_percent):
        self.current_image = self.image_processor.gaussian_noise(image=self.current_image, noise_percent=noise_percent)
        self.filename_suffix += f"_gauss{noise_percent}"
        return self

    def salt_pepper_noise(self, noise_percent):
        self.current_image = self.image_processor.salt_pepper_noise(image=self.current_image, noise_percent=noise_percent)
        self.filename_suffix += f"_salt{noise_percent}"
        return self

    def adjust_contrast(self, adjust_percent):
        self.current_image = self.image_processor.adjust_contrast(image=self.current_image, adjust_percent=adjust_percent)
        self.filename_suffix += f"_contrast{adjust_percent}"
        return self

    def adjust_bright(self, adjust_percent):
        self.current_image = self.image_processor.adjust_bright(image=self.current_image, adjust_percent=adjust_percent)
        self.filename_suffix += f"_bright{adjust_percent}"
        return self

    def save_image(self):
        filename = self.filename_suffix + self.image_suffix
        output_file = self.output_image_path / filename
        cv2.imwrite(str(output_file), self.current_image)
        return self

    def save_label(self):
        filename = self.filename_suffix + self.image_suffix
        output_file = self.output_label_path / filename
        with open(output_file, "w") as f:
            for label in self.current_labels:
                f.write(" ".join(label) + "\n")
        return self


In [ ]:
image_path = "input_images/example.jpg"
label_path = "input_labels/example.txt"
output_dir = "output"

(DataAugment(image_path, label_path, output_dir)
    .vertical()
    .gaussian_noise(noise_percent=15)
    .salt_pepper_noise(noise_percent=5)
    .adjust_bright(adjust_percent=20)
    .save_image()
    .save_label()
 )
print("증강된 이미지와 레이블이 자동 파일명과 함께 저장되었습니다.")
